## 4장: 밑바닥부터 GPT모델 구현하기

In [1]:
from importlib.metadata import version
print("matplotlib version: ", version("matplotlib"))
print("pytorch version: ", version("torch"))
print("tiktoken version: ", version("tiktoken"))

matplotlib version:  3.10.7
pytorch version:  2.6.0
tiktoken version:  0.11.0


In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,         # 어휘사전 크기
    "context_length": 1024,      # 문맥 길이  
    "emb_dim": 768,              # 임베딩 차원
    "n_heads": 12,               # 어텐션 헤드 개수
    "n_layers": 12,              # 층 개수
    "drop_rate": 0.1,            # dropout 비율
    "qkv_bias": False            # 쿼리, 키, 값을 만들때 편향 포함 여부
}

In [4]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        # Use a placeholder for layerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)
        
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds  = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        pass
    
    def forward(self, x):
        # 이 블록은 아무것도 하지 않고 입력을 그냥 반환합니다.
        return x
    
class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # 층 정규화 인터페이스를 흉내내기 위한 매개변수
    
    def forward(self, x):
        # 이 블록은 아무것도 하지 않고 입력을 그냥 반환
        return x